In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from typing import *
import numpy as np
from functools import partial
from fastprogress import progress_bar

import torch
from torch import Tensor, nn
import torch.nn.functional as F

from tomopt.volume import *
from tomopt.muon import *
from tomopt.inference import *
from tomopt.optimisation import *
from tomopt.core import *
from tomopt.utils import *
from tomopt.plotting import *
from tomopt.benchmarks.phi_detector import *

import seaborn as sns

In [3]:
DEVICE = torch.device("cpu")

In [4]:
def get_layers():
    layers = []
    lwh = Tensor([1,1,1])
    size = 0.1
    init_eff = 0.8
    init_res = 1000
    n_panels = 4
    layers.append(PanelDetectorLayer(pos='above', lw=lwh[:2], z=1, size=2*size,
                                     panels=[PhiDetectorPanel(init_phi=torch.rand(1)*2*torch.pi, res=init_res, eff=init_eff,
                                                              init_z=1-(i*(2*size)/n_panels), device=DEVICE) for i in range(n_panels)]))
    for z in [0.8,0.7,0.6,0.5,0.4,0.3]:
        layers.append(PassiveLayer(lw=lwh[:2], z=z, size=size, device=DEVICE))
    layers.append(PanelDetectorLayer(pos='below', lw=lwh[:2], z=0.2, size=2*size,
                                     panels=[PhiDetectorPanel(init_phi=torch.rand(1)*2*torch.pi, res=init_res, eff=init_eff,
                                                      init_z=0.2-(i*(2*size)/n_panels), device=DEVICE) for i in range(n_panels)]))

    return nn.ModuleList(layers) 

In [5]:
volume = Volume(get_layers())

In [6]:
gen = MuonGenerator2016.from_volume(volume, fixed_mom=None)

In [7]:
mu = MuonBatch(MuonResampler().resample(gen(10), volume, gen), init_z=1)

In [8]:
h = volume.get_detectors()[0].panels[0].get_hits(mu)

In [9]:
jacobian(h['reco_h'], volume.get_detectors()[0].panels[0].phi)

tensor([[0.1062],
        [0.2986],
        [0.1105],
        [0.1208],
        [0.1120],
        [0.6829],
        [0.2825],
        [0.0880],
        [0.6280],
        [0.5636]])

In [10]:
mu._hits

defaultdict(<function tomopt.muon.muon_batch.MuonBatch.__init__.<locals>.<lambda>()>,
            {})

In [11]:
mu.append_hits(volume.get_detectors()[0].panels[0].get_hits(mu), 'above')

In [12]:
h = mu._hits['above']['reco_h'];h

[tensor([[1.1593],
         [1.0333],
         [0.9000],
         [0.5426],
         [0.6701],
         [0.3646],
         [1.1741],
         [0.7729],
         [0.0984],
         [0.0287]], grad_fn=<AddBackward0>)]

In [13]:
h[0]

tensor([[1.1593],
        [1.0333],
        [0.9000],
        [0.5426],
        [0.6701],
        [0.3646],
        [1.1741],
        [0.7729],
        [0.0984],
        [0.0287]], grad_fn=<AddBackward0>)

In [14]:
jacobian(h[0], volume.get_detectors()[0].panels[0].phi)

tensor([[0.1062],
        [0.2986],
        [0.1105],
        [0.1208],
        [0.1120],
        [0.6829],
        [0.2825],
        [0.0880],
        [0.6280],
        [0.5636]])

In [15]:
volume.get_detectors()[0].scatter_and_propagate(mu, 0.1)

In [16]:
jacobian(h[0], volume.get_detectors()[0].panels[0].phi)

tensor([[0.1062],
        [0.2986],
        [0.1105],
        [0.1208],
        [0.1120],
        [0.6829],
        [0.2825],
        [0.0880],
        [0.6280],
        [0.5636]])

In [17]:
mu.append_hits(volume.get_detectors()[0].panels[1].get_hits(mu), 'above')

In [18]:
h = mu._hits['above']['reco_h'];h

[tensor([[1.1593],
         [1.0333],
         [0.9000],
         [0.5426],
         [0.6701],
         [0.3646],
         [1.1741],
         [0.7729],
         [0.0984],
         [0.0287]], grad_fn=<AddBackward0>),
 tensor([[-1.0067],
         [-0.8652],
         [-0.8532],
         [-0.5160],
         [-0.6190],
         [-0.1099],
         [-0.9790],
         [-0.6459],
         [ 0.1015],
         [ 0.1457]], grad_fn=<AddBackward0>)]

In [19]:
jacobian(h[0], volume.get_detectors()[0].panels[0].phi)

tensor([[0.1062],
        [0.2986],
        [0.1105],
        [0.1208],
        [0.1120],
        [0.6829],
        [0.2825],
        [0.0880],
        [0.6280],
        [0.5636]])

In [20]:
mu = MuonBatch(MuonResampler().resample(gen(10), volume, gen), init_z=1)

In [21]:
mu._hits

defaultdict(<function tomopt.muon.muon_batch.MuonBatch.__init__.<locals>.<lambda>()>,
            {})

In [22]:
volume.get_detectors()[0](mu)

In [23]:
mu._hits

defaultdict(<function tomopt.muon.muon_batch.MuonBatch.__init__.<locals>.<lambda>()>,
            {'above': defaultdict(list,
                         {'reco_h': [tensor([[1.2666],
                                   [0.4443],
                                   [0.2907],
                                   [0.5058],
                                   [0.1764],
                                   [0.9526],
                                   [1.0527],
                                   [0.3008],
                                   [0.3189],
                                   [0.1353]], grad_fn=<AddBackward0>),
                           tensor([[-1.0128],
                                   [-0.3406],
                                   [-0.2410],
                                   [-0.3913],
                                   [-0.2448],
                                   [-0.8258],
                                   [-0.8183],
                                   [-0.1490],
                    

In [24]:
h = mu._hits['above']['reco_h'];h

[tensor([[1.2666],
         [0.4443],
         [0.2907],
         [0.5058],
         [0.1764],
         [0.9526],
         [1.0527],
         [0.3008],
         [0.3189],
         [0.1353]], grad_fn=<AddBackward0>),
 tensor([[-1.0128],
         [-0.3406],
         [-0.2410],
         [-0.3913],
         [-0.2448],
         [-0.8258],
         [-0.8183],
         [-0.1490],
         [-0.1546],
         [ 0.0927]], grad_fn=<AddBackward0>),
 tensor([[ 1.0547],
         [ 0.4507],
         [ 0.2536],
         [ 0.4446],
         [-0.0328],
         [ 0.7480],
         [ 1.0492],
         [ 0.4251],
         [ 0.3907],
         [ 0.6369]], grad_fn=<AddBackward0>),
 tensor([[ 0.9970],
         [ 0.3577],
         [ 0.2547],
         [ 0.3875],
         [ 0.2491],
         [ 0.8687],
         [ 0.8245],
         [ 0.1329],
         [ 0.1117],
         [-0.0894]], grad_fn=<AddBackward0>)]

In [25]:
jacobian(h[0], volume.get_detectors()[0].panels[0].phi)

tensor([[ 0.4704],
        [ 0.2440],
        [ 0.1053],
        [ 0.2301],
        [-0.2173],
        [ 0.2521],
        [ 0.4901],
        [ 0.3585],
        [ 0.3471],
        [ 0.6228]])

In [26]:
def arb_rad_length(*,z:float, lw:Tensor, size:float) -> Tensor:
    rad_length = torch.ones(list((lw/size).long()))*X0['beryllium']
    if z >= 0.4 and z <= 0.5: rad_length[5:,5:] = X0['lead']
    return rad_length

In [27]:
volume.load_rad_length(arb_rad_length)

In [28]:
mu = MuonBatch(MuonResampler().resample(gen(100), volume, gen), init_z=1)
volume(mu)

In [29]:
hits = mu.get_hits()

In [30]:
mu.x.shape

torch.Size([100])

In [31]:
hits['above']['reco_h'].shape

torch.Size([100, 4, 1])

In [32]:
from tomopt.inference.scattering import AbsScatterBatch

class PhiDetScatterBatch(AbsScatterBatch):
    def _extract_hits(self) -> None:
        # reco x, reco y, gen z, must be a list to allow computation of uncertainty
        above_hits = torch.stack(
            [
                torch.cat([self.hits["above"]["reco_h"][:, i], self.hits["above"]["phi"][:, i], self.hits["above"]["z"][:, i]], dim=-1)
                for i in range(self.hits["above"]["reco_h"].shape[1])
            ],
            dim=1,
        )  # muons, panels, xyz
        below_hits = torch.stack(
            [
                torch.cat([self.hits["below"]["reco_h"][:, i], self.hits["below"]["phi"][:, i], self.hits["below"]["z"][:, i]], dim=-1)
                for i in range(self.hits["below"]["reco_h"].shape[1])
            ],
            dim=1,
        )
        _above_gen_hits = torch.stack(
            [torch.cat([self.hits["above"]["gen_xy"][:, i], self.hits["above"]["z"][:, i]], dim=-1) for i in range(self.hits["above"]["gen_xy"].shape[1])],
            dim=1,
        )  # muons, panels, xyz
        _below_gen_hits = torch.stack(
            [torch.cat([self.hits["below"]["gen_xy"][:, i], self.hits["below"]["z"][:, i]], dim=-1) for i in range(self.hits["below"]["gen_xy"].shape[1])],
            dim=1,
        )
        self._n_hits_above = above_hits.shape[1]
        self._n_hits_below = below_hits.shape[1]

        # Combine all input vars into single tensor, NB ideally would stack to new dim but can't assume same number of panels above & below
        self._reco_hits = torch.cat((above_hits, below_hits), dim=1)  # muons, all panels, reco h,phi,z
        self._gen_hits = torch.cat((_above_gen_hits, _below_gen_hits), dim=1)  # muons, all panels, true xyz

    def plot_scatter(self, idx: int) -> None:
        raise NotImplementedError("Ah, I see you've just volunteered to implement this!")

    @staticmethod
    def _get_hit_uncs(zordered_panels: List[PhiDetectorPanel], hits: Tensor) -> Tensor:
        uncs: List[Tensor] = []
        for l, h in zip(zordered_panels, hits.unbind(1)):
            r = 1 / l.resolution.expand(len(h), 1)
            uncs.append(torch.cat([r, torch.zeros((len(h), 2), device=r.device)], dim=-1))
        return torch.stack(uncs, dim=1)  # muons, panels, unc h,phi,z, zero unc for phi and z
    

    def _compute_tracks(self) -> None:
        def _get_panels() -> List[PhiDetectorPanel]:
            panels = []
            for det in self.volume.get_detectors():
                if not isinstance(det, PanelDetectorLayer):
                    raise ValueError(f"Detector {det} is not a PanelDetectorLayer")
                panels += [det.panels[j] for j in det.get_panel_zorder()]
            return panels

        self._hit_uncs = self._get_hit_uncs(_get_panels(), self.gen_hits)
#         self._track_in, self._track_start_in = self.get_muon_trajectory(self.above_hits, self.above_hit_uncs, self.volume.lw)
#         self._track_out, self._track_start_out = self.get_muon_trajectory(self.below_hits, self.below_hit_uncs, self.volume.lw)

    def _compute_scatters(self) -> None:
        r"""
        Currently only handles detectors above and below passive volume

        Scatter locations adapted from:
        @MISC {3334866,
            TITLE = {Closest points between two lines},
            AUTHOR = {Brian (https://math.stackexchange.com/users/72614/brian)},
            HOWPUBLISHED = {Mathematics Stack Exchange},
            NOTE = {URL:https://math.stackexchange.com/q/3334866 (version: 2019-08-26)},
            EPRINT = {https://math.stackexchange.com/q/3334866},
            URL = {https://math.stackexchange.com/q/3334866}
        }
        """

        self._extract_hits()
        self._compute_tracks()
#         self._filter_scatters()

#         # Track computations
#         self._cross_track = torch.cross(self.track_in, self.track_out, dim=1)  # connecting vector perpendicular to both lines

#         rhs = self.track_start_out - self.track_start_in
#         lhs = torch.stack([self.track_in, -self.track_out, self._cross_track], dim=1).transpose(2, 1)
#         # coefs = torch.linalg.solve(lhs, rhs)  # solve p1+t1*v1 + t3*v3 = p2+t2*v2 => p2-p1 = t1*v1 - t2*v2 + t3*v3
#         self._track_coefs = (lhs.inverse() @ rhs[:, :, None]).squeeze(-1)

#     @staticmethod
#     def get_muon_trajectory(hits: Tensor, uncs: Tensor, lw: Tensor) -> Tuple[Tensor, Tensor]:
#         r"""
#         hits = (muons,panels,(h,phi,z))
#         uncs = (muons,panels,(unc,0,0))

#         Assumes no uncertainty for z and phi

#         Uses an analytic likelihood-maximisation: L = \prod_i G[h_i - h_{i,opt}(x0,y0,theta,phi)]
#         where:
#         h_{i,opt} = x_{i,opt} cos(phi_i) + y_{i,opt} sin(phi_i)
#         x_{i,opt} = x0 + z_i tan(theta) cos(phi)
#         y_{i,opt} = y0 + z_i tan(theta) sin(phi)

#         x0, y0, z0 = track coordinates at z=0
#         theta, phi = track angles

#         x0 = sum_i[ ((cos(phi_i)/(unc_i^2))*(h_i-(z_i*tan(theta)*cos(phi)*cos(phi_i))-(sin(phi_i)*(y0+(z_i*tan(theta)*sin(phi))))) ] / sum_i[ ((cos(phi_i)^2)/(unc_i^2)) ]
#         y0 = sum_i[ ((sin(phi_i)/(unc_i^2))*(h_i-(z_i*tan(theta)*sin(phi)*cos(phi_i))-(cos(phi_i)*(x0+(z_i*tan(theta)*sin(phi))))) ] / sun_i[ ((sin(phi_i)^2)/(unc_i^2)) ]
#         theta = tan^-1[ sum_i[ (z_i/(unc_i^2))*((h_i*((cos(phi)*cos(phi_i))+(sin(phi)*sin(phi_i))))-(x0*((cos(phi_i)*sin(phi_i)*sin(phi))+(cos(phi)*cos(phi_i)*cos(phi_i))))-(y0((cos(phi)*cos(phi_i)*sin(phi_i))+(sin(phi)*sin(phi_i)*sin(phi_i))))) ] / sum_i[ ((z_i^2)/(unc_i^2))*((cos(phi)*cos(phi_i))+(sin(phi)*sin(sin(phi_i)))) ] ]
#         phi = sin^-1[ ((z_i*sin(phi_i))/(unc_i^2))*(h_i-(y0*sin(phi_i))-(x0*cos(phi_i))-(z_i*tan(theta)*cos(phi)*cos(phi_i))) ] / sum_i[ (((z_i*sin(phi_i))^2)/(unc_i^2))*tan(theta) ]

#         In eval mode:
#             Muons with <2 hits within panels have NaN trajectory.
#             Muons with >=2 hits in panels have valid trajectories
#         """

#         hits = torch.where(torch.isinf(hits), lw.mean().type(hits.type()) / 2, hits)
#         uncs = torch.nan_to_num(uncs)  # Set Infs to large number

#         x0: Tensor
#         y0: Tensor  # Track positions at Z0=0
#         phi: Tensor
#         theta: Tensor  # Track angles

In [33]:
sb = PhiDetScatterBatch(mu, volume)

In [34]:
sb.above_hits.shape, sb.above_hit_uncs.shape

(torch.Size([100, 4, 3]), torch.Size([100, 4, 3]))

In [35]:
volume.get_detectors()[0].panels[0].phi

Parameter containing:
tensor(0.1366, requires_grad=True)

In [36]:
jacobian(sb.above_hits, volume.get_detectors()[0].panels[0].phi)

tensor([[[0.1394, 1.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000]],

        [[0.8318, 1.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000]],

        [[0.8861, 1.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000]],

        ...,

        [[0.7954, 1.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000]],

        [[0.1217, 1.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000]],

        [[0.5174, 1.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000]]])

In [37]:
hits = sb.above_hits
uncs = sb.above_hit_uncs

In [38]:
hits = torch.where(torch.isinf(hits), volume.lw.mean().type(hits.type()) / 2, hits)
uncs = torch.nan_to_num(uncs)  # Set Infs to large number

In [39]:
uncs2 = uncs[:,:,0:1]**2

In [40]:
track = torch.ones(len(hits),1,4, device=hits.device, requires_grad=True)/2

In [41]:
def get_nll(hits:Tensor, uncs2:Tensor, track:Tensor) -> Tensor:
    x0 = track[:,:,0:1]
    y0 = track[:,:,1:2]
    phi = track[:,:,2:3]
    theta = track[:,:,3:4]
    h_i = hits[:,:,0:1]
    phi_i = hits[:,:,1:2]
    z_i = hits[:,:,2:3]

    x_i_opt = x0 + (z_i*theta.tan()*phi.cos())
    y_i_opt = y0 + (z_i*theta.tan()*phi.sin())
    h_i_opt = (x_i_opt*phi_i.cos()) + (y_i_opt*phi_i.sin())  

    return (0.5*((h_i-h_i_opt).square()/uncs2).sum(1))-(1/(2*torch.pi*uncs2).sqrt().sum(1))

# Newton

In [79]:
nll = get_nll(hits, uncs2, track)
nll.shape

torch.Size([100, 1])

In [80]:
grad = jacobian(nll, track, create_graph=True).sum((2,3))

In [46]:
grad.shape

torch.Size([10, 1, 4])

In [47]:
grad[0]

tensor([[  45554.0938, -106952.0938,  -60050.1172,  -20507.5391]],
       grad_fn=<SelectBackward0>)

In [48]:
hesse = jacobian(grad, track, create_graph=True).sum((3,4))

In [49]:
hesse.shape

torch.Size([10, 1, 4, 4])

In [50]:
(grad@hesse.inverse()).shape

torch.Size([10, 10, 1, 4])

In [51]:
(grad.unsqueeze(1)@hesse.inverse()).squeeze(1).shape

torch.Size([10, 1, 4])

In [52]:
step = (grad.unsqueeze(1)@hesse.inverse()).squeeze(1)

In [53]:
step.mean(0)

tensor([[4.0454e-01, 2.7361e-01, 2.1547e-04, 6.3728e-04]],
       grad_fn=<MeanBackward1>)

In [54]:
lr = Tensor([[[0.1,0.1,100,10]]])

In [55]:
lr*step.mean(0)

tensor([[[0.0405, 0.0274, 0.0215, 0.0064]]], grad_fn=<MulBackward0>)

In [81]:
lr = Tensor([[[0.1,0.1,100,10]]])
track = torch.ones(len(hits),1,4, device=hits.device, requires_grad=True)/2

for i in progress_bar(range(100)):  # Newton optimise nuisances
    nll = get_nll(hits, uncs2, track)
    grad = jacobian(nll, track, create_graph=True).sum((2,3))
    hesse = jacobian(grad, track, create_graph=True).sum((3,4))
    step = lr*(grad.unsqueeze(1)@hesse.inverse()).squeeze(1)
    track = track-step
    if i > 0 and i % 10 == 0:
            lr = lr / 2
    print(nll.mean(0).data, track.mean(0).data)

tensor([552313.7500]) tensor([[0.4526, 0.4739, 0.4703, 0.4935]])
tensor([459080.0938]) tensor([[0.4125, 0.4534, 0.4413, 0.4841]])
tensor([366024.0938]) tensor([[0.3781, 0.4370, 0.3915, 0.4724]])
tensor([299159.9688]) tensor([[0.3505, 0.4242, 0.3703, 0.4530]])
tensor([254436.2812]) tensor([[0.3306, 0.4144, 0.3774, 0.4344]])
tensor([206660.5156]) tensor([[0.3156, 0.4069, 0.3972, 0.4153]])
tensor([175511.4062]) tensor([[0.3051, 0.4011, 0.4483, 0.3922]])
tensor([132246.6562]) tensor([[0.2990, 0.3972, 0.5605, 0.3667]])
tensor([103370.5234]) tensor([[0.2959, 0.3939, 0.5222, 0.3501]])
tensor([87922.6016]) tensor([[0.2956, 0.3934, 0.6164, 0.3302]])
tensor([89197.1016]) tensor([[0.2984, 0.3923, 0.7041, 0.3203]])
tensor([79813.7188]) tensor([[0.3012, 0.3917, 0.7114, 0.3158]])
tensor([73537.2500]) tensor([[0.3044, 0.3911, 0.7800, 0.3090]])
tensor([63532.7539]) tensor([[0.3075, 0.3907, 0.8599, 0.2989]])
tensor([63752.5508]) tensor([[0.3112, 0.3904, 0.9420, 0.2865]])
tensor([71545.9375]) tensor([[0

KeyboardInterrupt: 

In [64]:
%%time
jac = jacobian(track, volume.get_detectors()[0].panels[0].phi)

CPU times: user 7.71 s, sys: 931 ms, total: 8.65 s
Wall time: 4.38 s


In [62]:
jac.shape

torch.Size([10, 1, 4])

# GD

In [42]:
lr = Tensor([[[0.1,0.1,100,10]]])
track = torch.ones(len(hits),1,4, device=hits.device, requires_grad=True)/2
nll = get_nll(hits, uncs2, track)

In [55]:
lr = 1e-7
track = torch.ones(len(hits),1,4, device=hits.device, requires_grad=True)/2

for i in progress_bar(range(100)):  # Newton optimise nuisances
    nll = get_nll(hits, uncs2, track)
    grad = jacobian(nll, track, create_graph=True).sum((2,3))
    step = lr*grad
    track = track-step
    if i > 0 and i % 10 == 0:
            lr = lr / 2
    print(nll.mean(0).data, grad.mean(0).data, track.mean(0).data)

tensor([567713.7500]) tensor([[1401283.0000,  317070.5312, -204165.4531, 1678142.3750]]) tensor([[0.3599, 0.4683, 0.5204, 0.3322]])
tensor([110723.5391]) tensor([[455298.6250, 173513.0000,  -5944.2505, 478935.1875]]) tensor([[0.3143, 0.4509, 0.5210, 0.2843]])
tensor([53187.8008]) tensor([[184512.6406, 124359.8516,  17791.1094, 210955.7656]]) tensor([[0.2959, 0.4385, 0.5192, 0.2632]])
tensor([35892.2422]) tensor([[ 71192.4688,  98165.2812,  21492.1582, 100994.7734]]) tensor([[0.2888, 0.4287, 0.5171, 0.2531]])
tensor([27552.3906]) tensor([[22116.1680, 81998.2500, 20489.3652, 51755.2617]]) tensor([[0.2866, 0.4205, 0.5150, 0.2479]])
tensor([22043.0176]) tensor([[ 1134.9061, 70754.4766, 18359.3418, 28855.1016]]) tensor([[0.2864, 0.4134, 0.5132, 0.2450]])
tensor([17893.1621]) tensor([[-7373.6738, 62160.6992, 16103.8398, 17833.7930]]) tensor([[0.2872, 0.4072, 0.5116, 0.2433]])
tensor([14632.6279]) tensor([[-10364.4785,  55148.2266,  14018.3174,  12284.0811]]) tensor([[0.2882, 0.4017, 0.5102, 

tensor([2224.5042]) tensor([[-3717.6082, 12643.7812,  2327.3804,  2547.4131]]) tensor([[0.2972, 0.3662, 0.5024, 0.2359]])
tensor([2223.4038]) tensor([[-3715.3765, 12633.6240,  2325.1982,  2546.2217]]) tensor([[0.2973, 0.3662, 0.5024, 0.2359]])
tensor([2222.3054]) tensor([[-3713.1572, 12623.4697,  2323.0203,  2545.0198]]) tensor([[0.2973, 0.3662, 0.5024, 0.2359]])
tensor([2221.2083]) tensor([[-3710.9343, 12613.3291,  2320.8433,  2543.8274]]) tensor([[0.2973, 0.3662, 0.5024, 0.2359]])
tensor([2220.1135]) tensor([[-3708.7078, 12603.1963,  2318.6675,  2542.6396]]) tensor([[0.2973, 0.3662, 0.5024, 0.2359]])
tensor([2219.0203]) tensor([[-3706.4990, 12593.0586,  2316.4941,  2541.4341]]) tensor([[0.2973, 0.3661, 0.5024, 0.2359]])
tensor([2218.4744]) tensor([[-3705.3843, 12588.0029,  2315.4087,  2540.8462]]) tensor([[0.2973, 0.3661, 0.5024, 0.2359]])
tensor([2217.9287]) tensor([[-3704.2844, 12582.9414,  2314.3245,  2540.2427]]) tensor([[0.2973, 0.3661, 0.5024, 0.2359]])
tensor([2217.3838]) tens

In [ ]:
%%time
jac = jacobian(track, volume.get_detectors()[0].panels[0].phi)

In [62]:
jac.shape

torch.Size([10, 1, 4])